# DB2-Salesforce connector: pull in new email flags from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"shit9927289sSYTkLiGvyK2UrazHFmjFUGU",#GqnyixN6W3zVDAFXk80lgMAR4
} 

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Get the latest email opt in/out data from DB2


In [3]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# get new preferences for users
pcomms = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('personalizedcommunication') order by rand() limit 50000;",engine) #and modifiedDate >= '2019-08-08' 

up_news = pd.read_sql_query("select id, user_id, profile_key, profile_value from jos_user_profiles \
where profile_key in ('updates_news') order by rand() limit 50000;",engine)

In [4]:
display(pcomms.head(2))
display(pcomms.tail(2))
display(up_news.head(2))
display(up_news.tail(2))

,id,user_id,profile_key,profile_value
0,1798861,162479,personalizedcommunication,"Yes, I would like to receive personalized comm..."
1,1906955,260680,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
49998,1637079,60477,personalizedcommunication,"Yes, I would like to receive personalized comm..."
49999,1792365,158390,personalizedcommunication,"Yes, I would like to receive personalized comm..."


,id,user_id,profile_key,profile_value
0,1593640,38367,updates_news,"No, I do not want to receive updates and news"
1,1827364,181427,updates_news,"No, I do not want to receive updates and news"


,id,user_id,profile_key,profile_value
49998,1942102,286426,updates_news,"No, I do not want to receive updates and news"
49999,1923630,273776,updates_news,"No, I do not want to receive updates and news"


In [5]:
## adjusting new_email dataframe for the new email optin values 
# 0 or 1
pcomms_t = pcomms[['user_id','profile_value']]
up_news_t = up_news[['user_id','profile_value']]

display(pcomms_t.head(2))
display(up_news_t.head(2))


,user_id,profile_value
0,162479,"Yes, I would like to receive personalized comm..."
1,260680,"Yes, I would like to receive personalized comm..."


,user_id,profile_value
0,38367,"No, I do not want to receive updates and news"
1,181427,"No, I do not want to receive updates and news"


In [6]:
## filtering pcomms_t with 0 or 1
ptt = pcomms_t['profile_value'].to_list()

for index, val in enumerate(ptt):
    val = val.split(',')[0]
    if val == 'Yes':
        ptt[index] = 1 # 1 indicates opt-in
    else:
        ptt[index] = 0 # 1 indicates opt-out

pcomms_t.loc[:,'profile_value'] = ptt


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
## filtering up news t with 0 or 1
untt = up_news_t['profile_value'].to_list()

for index, val in enumerate(untt):
    val = val.split(',')[0]
    if val == 'Yes':
        untt[index] = 1 # 1 indicates opt-in
    else:
        untt[index] = 0 # 1 indicates opt-out

up_news_t.loc[:,'profile_value'] = untt


In [8]:
## display
display(pcomms_t.head(2))
display(pcomms_t.tail(2))
display(up_news_t.head(2))
display(up_news_t.tail(2))

,user_id,profile_value
0,162479,1
1,260680,1


,user_id,profile_value
49998,60477,1
49999,158390,1


,user_id,profile_value
0,38367,0
1,181427,0


,user_id,profile_value
49998,286426,0
49999,273776,0


In [9]:
## create new dataframe
new_opts = pd.DataFrame()

new_opts['nanoHUB_user_ID__c'] = pcomms_t['user_id']
new_opts['personalizedcommunication__c'] = pcomms_t['profile_value']
new_opts['updatesnews__c'] = up_news_t['profile_value']


In [10]:
display(new_opts.head(2))
print(type(new_opts))

,nanoHUB_user_ID__c,personalizedcommunication__c,updatesnews__c
0,162479,1,0
1,260680,1,0


<class 'pandas.core.frame.DataFrame'>


## Send to SF

In [14]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)


Obtained Salesforce access token ...... True


In [15]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(new_opts)


[Success] Bulk job creation successful. Job ID = 7505w00000QNT5AAAX
hello
[Success] CSV upload successful. Job ID = 7505w00000QNT5AAAX
[Success] Closing job successful. Job ID = 7505w00000QNT5AAAX


In [18]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000QNT5AAAX',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-09-27T23:58:30.000+0000',
 'systemModstamp': '2020-09-28T00:00:32.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 50000,
 'numberRecordsFailed': 36,
 'retries': 0,
 'totalProcessingTime': 485746,
 'apiActiveProcessingTime': 438708,
 'apexProcessingTime': 4260084}

In [19]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",nanoHUB_user_ID__c,personalizedcommunication__c,updatesnews__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","105411.0","true","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","300187.0","false","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","298538.0","true","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","297246.0","true","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","299670.0","true","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","299325.0","false","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","300595.0","false","false"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","297476.0","true"